<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/GogoVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/content/drive/MyDrive/MIASUnited

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnited'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split
)

# Load and augment data
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Specify training subset
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Specify validation subset
)

# Define the base model (VGG16 without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), #Best Hyperparameters:  {'learning_rate': 0.001}
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs
    validation_data=validation_generator,
    verbose=1
)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Ensure no shuffling for correct evaluation
)

test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')


Found 90 images belonging to 2 classes.
Found 22 images belonging to 2 classes.
Epoch 1/10
3/3 [==============================] - 114s 36s/step - loss: 0.7109 - accuracy: 0.5778 - val_loss: 0.6846 - val_accuracy: 0.8182
Epoch 2/10
3/3 [==============================] - 81s 29s/step - loss: 0.6118 - accuracy: 0.8222 - val_loss: 0.4788 - val_accuracy: 0.8182
Epoch 3/10
3/3 [==============================] - 83s 32s/step - loss: 0.4374 - accuracy: 0.8333 - val_loss: 0.5189 - val_accuracy: 0.8636
Epoch 4/10
3/3 [==============================] - 81s 29s/step - loss: 0.4414 - accuracy: 0.8778 - val_loss: 0.4240 - val_accuracy: 0.8182
Epoch 5/10
3/3 [==============================] - 79s 30s/step - loss: 0.3550 - accuracy: 0.8667 - val_loss: 0.4299 - val_accuracy: 0.8182
Epoch 6/10
3/3 [==============================] - 76s 26s/step - loss: 0.3692 - accuracy: 0.8444 - val_loss: 0.3812 - val_accuracy: 0.8182
Epoch 7/10
3/3 [==============================] - 82s 29s/step - loss: 0.3407 - accur

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnits'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (VGG16 without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    # Split the data
    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    # Use the training data for both training and validation generators
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices for validation too
        shuffle=True
    )

    # Build the model
    model = build_model()

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,  # Adjust the number of epochs
        validation_data=validation_generator,
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/10
4/4 [==============================] - 166s 45s/step - loss: 0.7202 - accuracy: 0.5982 - val_loss: 0.6861 - val_accuracy: 0.5625
Epoch 2/10
4/4 [==============================] - 149s 43s/step - loss: 0.7042 - accuracy: 0.5268 - val_loss: 0.6634 - val_accuracy: 0.5536
Epoch 3/10
4/4 [==============================] - 159s 50s/step - loss: 0.6575 - accuracy: 0.6161 - val_loss: 0.6604 - val_accuracy: 0.6071
Epoch 4/10
4/4 [==============================] - 156s 48s/step - loss: 0.6644 - accuracy: 0.5893 - val_loss: 0.6394 - val_accuracy: 0.5893
Epoch 5/10
4/4 [==============================] - 152s 42s/step - loss: 0.6483 - accuracy: 0.6250 - val_loss: 0.6914 - val_accuracy: 0.4911
Epoch 6/10
4/4 [==============================] - 156s 45s/step - loss: 0.6852 - accuracy: 0.5357 - val

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnited'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (VGG16 without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    # Split the data
    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    # Use the training data for both training and validation generators
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices for validation too
        shuffle=True
    )

    # Build the model
    model = build_model()

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,  # Adjust the number of epochs
        validation_data=validation_generator,
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/10
4/4 [==============================] - 97s 28s/step - loss: 0.6697 - accuracy: 0.8214 - val_loss: 0.5188 - val_accuracy: 0.8214
Epoch 2/10
4/4 [==============================] - 90s 25s/step - loss: 0.5041 - accuracy: 0.8304 - val_loss: 0.5074 - val_accuracy: 0.8214
Epoch 3/10
4/4 [==============================] - 90s 26s/step - loss: 0.4321 - accuracy: 0.8304 - val_loss: 0.3896 - val_accuracy: 0.8304
Epoch 4/10
4/4 [==============================] - 93s 27s/step - loss: 0.4044 - accuracy: 0.8304 - val_loss: 0.3855 - val_accuracy: 0.8393
Epoch 5/10
4/4 [==============================] - 126s 38s/step - loss: 0.3819 - accuracy: 0.8482 - val_loss: 0.3553 - val_accuracy: 0.8839
Epoch 6/10
4/4 [==============================] - 88s 25s/step - loss: 0.3646 - accuracy: 0.8839 - val_loss